In [1]:
# Import required Libraries
import tkinter as tk
from tkinter import *
from PIL import Image
from PIL import ImageTk
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

In [2]:
# Build the convolution network architecture
face_model = Sequential()
face_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
face_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))
face_model.add(Flatten())
face_model.add(Dense(1024, activation='relu'))
face_model.add(Dropout(0.5))
face_model.add(Dense(7, activation='softmax'))

In [3]:
# Load the saved weights
face_model.load_weights('recognition_model.h5')

In [4]:
# Disable OpenCL
cv2.ocl.setUseOpenCL(False)

In [5]:
# Create Datasets Dictionaries
facial_dict = {0: "   Angry   ", 1: "Disgusted", 2: "  Fearful  ", 3: "   Happy   ", 4: "  Neutral  ", 5: "    Sad    ", 6: "Surprised"}
#emojis_dict = {0:"train/angry", 1:"train/disgusted", 2:"train/fearful", 3:"train/happy", 4:"train/neutral", 5:"train/sad", 6:"train/surprised"}
emojis_dict = {
    0: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\angry.png",
    1: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\disgust.png",
    2: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\fear.png",
    3: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\happy.png",
    4: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\neutral.png",
    5: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\sad.png",
    6: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\surprise.png"
}



In [6]:
# Global variables
global last_frame1
last_frame1 = np.zeros((480, 640, 3), dtype=np.uint8)
global cap1
show_text=[0]

In [7]:
# Function to get face captured and recognize emotion
def Capture_Image():
    global cap1
    cap1 = cv2.VideoCapture(0)
    if not cap1.isOpened():
        print("cant open the camera1")
    global frame_number
    length =int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_number+=1
    if frame_number>=length:
        exit()
    cap1.set(1,frame_number)
    flag1, frame1 = cap1.read()
    frame1 = cv2.resize(frame1,(600,500))
    # It will detect the face in the video and bound it with a rectangular box
    #bound_box = cv2.CascadeClassifier('haarcascades_cuda/haarcascade_frontalface_default.xml')
    bounding_box = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    n_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in n_faces:
        cv2.rectangle(frame1, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_frame = gray_frame[y:y + h, x:x + w]
        crop_img = np.expand_dims(np.expand_dims(cv2.resize(roi_frame, (48, 48)), -1), 0)
        prediction = face_model.predict(crop_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame1, facial_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        show_text[0]=maxindex

    if flag1 is None:
        print ("Error!")

    elif flag1:
        global last_frame1
        last_frame1 = frame1.copy()
        pic = cv2.cvtColor(last_frame1, cv2.COLOR_BGR2RGB) #to store the image
        img = Image.fromarray(pic)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        lmain.after(10, Capture_Image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit()

In [8]:
# Function to show emoji according to facial expression
def Get_Emoji():
    frame2 = cv2.imread(emojis_dict[show_text[0]])
    if frame2 is None:
        print("Error: Could not load emoji image.")
        return
    pic2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
    img2 = Image.fromarray(pic2)
    imgtk2 = ImageTk.PhotoImage(image=img2)
    lmain2.imgtk2 = imgtk2
    lmain3.configure(text=facial_dict[show_text[0]], font=('arial', 45, 'bold'))
    lmain2.configure(image=imgtk2)
    root.update()
    lmain2.after(10, Get_Emoji)
if cv2.waitKey(1) & 0xFF == ord('q'):
    exit()

In [9]:
# GUI Window to show captured image with emoji
if __name__ == '__main__':
    frame_number=0
    root=tk.Tk()
    heading = Label(root,bg='black')
    heading.pack()
    heading2=Label(root,text="Face To Emojify",pady=20, font=('arial',45,'bold'),bg='black',fg='#CDCDCD')#to label the output
    heading2.pack()
    lmain = tk.Label(master=root,padx=50,bd=10)
    lmain2 = tk.Label(master=root,bd=10)
    lmain3=tk.Label(master=root,bd=10,fg="#CDCDCD",bg='black')
    lmain.pack(side=LEFT)
    lmain.place(x=50,y=250)
    lmain3.pack()
    lmain3.place(x=960,y=250)
    lmain2.pack(side=RIGHT)
    lmain2.place(x=900,y=350)
    root.title("Face To Emojify")
    root.geometry("1400x900+100+10")
    root['bg']='black'
    exitbutton = Button(root, text='Quit',fg="red",command=root.destroy,font=('arial',25,'bold')).pack(side = BOTTOM)
    Capture_Image()
    Get_Emoji()
    root.mainloop()

1/1 [==============================] - 0s 19ms/step


In [1]:
# Import required Libraries
import tkinter as tk
from tkinter import *
from PIL import Image
from PIL import ImageTk
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D



# Build the convolution network architecture
face_model = Sequential()
face_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
face_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))
face_model.add(Flatten())
face_model.add(Dense(1024, activation='relu'))
face_model.add(Dropout(0.5))
face_model.add(Dense(7, activation='softmax'))


# Load the saved weights
face_model.load_weights('recognition_model.h5')



# Disable OpenCL
cv2.ocl.setUseOpenCL(False)



# Create Datasets Dictionaries
facial_dict = {0: "   Angry   ", 1: "Disgusted", 2: "  Fearful  ", 3: "   Happy   ", 4: "  Neutral  ", 5: "    Sad    ", 6: "Surprised"}
#emojis_dict = {0:"train/angry", 1:"train/disgusted", 2:"train/fearful", 3:"train/happy", 4:"train/neutral", 5:"train/sad", 6:"train/surprised"}
emojis_dict = {
    0: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\angry.png",
    1: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\disgust.png",
    2: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\fear.png",
    3: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\happy.png",
    4: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\neutral.png",
    5: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\sad.png",
    6: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\surprise.png"
}



# Global variables
global last_frame1
last_frame1 = np.zeros((480, 640, 3), dtype=np.uint8)
global cap1
show_text=[0]



# Function to get face captured and recognize emotion
def Capture_Image():
    global cap1
    cap1 = cv2.VideoCapture(0)
    if not cap1.isOpened():
        print("cant open the camera1")
    global frame_number
    length =int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_number+=1
    if frame_number>=length:
        exit()
    cap1.set(1,frame_number)
    flag1, frame1 = cap1.read()
    frame1 = cv2.resize(frame1,(600,500))
    # It will detect the face in the video and bound it with a rectangular box
    #bound_box = cv2.CascadeClassifier('haarcascades_cuda/haarcascade_frontalface_default.xml')
    bounding_box = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    n_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in n_faces:
        cv2.rectangle(frame1, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_frame = gray_frame[y:y + h, x:x + w]
        crop_img = np.expand_dims(np.expand_dims(cv2.resize(roi_frame, (48, 48)), -1), 0)
        prediction = face_model.predict(crop_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame1, facial_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        show_text[0]=maxindex

    if flag1 is None:
        print ("Error!")

    elif flag1:
        global last_frame1
        last_frame1 = frame1.copy()
        pic = cv2.cvtColor(last_frame1, cv2.COLOR_BGR2RGB) #to store the image
        img = Image.fromarray(pic)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        lmain.after(10, Capture_Image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit()
        

# Function to show emoji according to facial expression
def Get_Emoji():
    frame2 = cv2.imread(emojis_dict[show_text[0]])
    if frame2 is None:
        print("Error: Could not load emoji image.")
        return
    pic2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
    img2 = Image.fromarray(pic2)
    imgtk2 = ImageTk.PhotoImage(image=img2)
    lmain2.imgtk2 = imgtk2
    lmain3.configure(text=facial_dict[show_text[0]], font=('arial', 45, 'bold'))
    lmain2.configure(image=imgtk2)
    root.update()
    lmain2.after(10, Get_Emoji)
if cv2.waitKey(1) & 0xFF == ord('q'):
    exit()
    
    
    


# GUI Window to show captured image with emoji
if __name__ == '__main__':
    frame_number=0
    root=tk.Tk()
    heading = Label(root,bg='black')
    heading.pack()
    heading2=Label(root,text="Face To Emojify",pady=20, font=('arial',45,'bold'),bg='black',fg='#CDCDCD')#to label the output
    heading2.pack()
    lmain = tk.Label(master=root,padx=50,bd=10)
    lmain2 = tk.Label(master=root,bd=10)
    lmain3=tk.Label(master=root,bd=10,fg="#CDCDCD",bg='black')
    lmain.pack(side=LEFT)
    lmain.place(x=50,y=250)
    lmain3.pack()
    lmain3.place(x=960,y=250)
    lmain2.pack(side=RIGHT)
    lmain2.place(x=900,y=250)
    root.title("Face To Emojify")
    root.geometry("1400x900+100+10")
    root['bg']='black'
    exitbutton = Button(root, text='Quit',fg="red",command=root.destroy,font=('arial',25,'bold')).pack(side = BOTTOM)
    Capture_Image()
    Get_Emoji()
    root.mainloop()



1/1 [==============================] - 0s 47ms/step


In [1]:
# Import required Libraries
import tkinter as tk
from tkinter import *
from PIL import Image
from PIL import ImageTk
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D



# Build the convolution network architecture
face_model = Sequential()
face_model.add(Conv2D(32, kernel_size=(3, 3), activation='softmax', input_shape=(48,48,1)))
face_model.add(Conv2D(64, kernel_size=(3, 3), activation='softmax'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='softmax'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='softmax'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))
face_model.add(Flatten())
face_model.add(Dense(1024, activation='softmax'))
face_model.add(Dropout(0.5))
face_model.add(Dense(7, activation='softmax'))


# Load the saved weights
face_model.load_weights('recognition_model.h5')



# Disable OpenCL
cv2.ocl.setUseOpenCL(False)



# Create Datasets Dictionaries
facial_dict = {0: "   Angry   ", 1: "Disgusted", 2: "  Fearful  ", 3: "   Happy   ", 4: "  Neutral  ", 5: "    Sad    ", 6: "Surprised"}
#emojis_dict = {0:"train/angry", 1:"train/disgusted", 2:"train/fearful", 3:"train/happy", 4:"train/neutral", 5:"train/sad", 6:"train/surprised"}
emojis_dict = {
    0: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\angry.png",
    1: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\disgust.png",
    2: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\fear.png",
    3: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\happy.png",
    4: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\neutral.png",
    5: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\sad.png",
    6: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\surprise.png"
}



# Global variables
global last_frame1
last_frame1 = np.zeros((480, 640, 3), dtype=np.uint8)
global cap1
show_text=[0]



# Function to get face captured and recognize emotion
def Capture_Image():
    global cap1
    cap1 = cv2.VideoCapture(0)
    if not cap1.isOpened():
        print("cant open the camera1")
    global frame_number
    length =int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_number+=1
    if frame_number>=length:
        exit()
    cap1.set(1,frame_number)
    flag1, frame1 = cap1.read()
    frame1 = cv2.resize(frame1,(600,500))
    # It will detect the face in the video and bound it with a rectangular box
    #bound_box = cv2.CascadeClassifier('haarcascades_cuda/haarcascade_frontalface_default.xml')
    bounding_box = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    n_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in n_faces:
        cv2.rectangle(frame1, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_frame = gray_frame[y:y + h, x:x + w]
        crop_img = np.expand_dims(np.expand_dims(cv2.resize(roi_frame, (48, 48)), -1), 0)
        prediction = face_model.predict(crop_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame1, facial_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        show_text[0]=maxindex

    if flag1 is None:
        print ("Error!")

    elif flag1:
        global last_frame1
        last_frame1 = frame1.copy()
        pic = cv2.cvtColor(last_frame1, cv2.COLOR_BGR2RGB) #to store the image
        img = Image.fromarray(pic)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        lmain.after(10, Capture_Image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit()
        

# Function to show emoji according to facial expression
def Get_Emoji():
    frame2 = cv2.imread(emojis_dict[show_text[0]])
    if frame2 is None:
        print("Error: Could not load emoji image.")
        return
    pic2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
    img2 = Image.fromarray(pic2)
    imgtk2 = ImageTk.PhotoImage(image=img2)
    lmain2.imgtk2 = imgtk2
    lmain3.configure(text=facial_dict[show_text[0]], font=('arial', 45, 'bold'))
    lmain2.configure(image=imgtk2)
    root.update()
    lmain2.after(10, Get_Emoji)
if cv2.waitKey(1) & 0xFF == ord('q'):
    exit()
    
    
    


# GUI Window to show captured image with emoji
if __name__ == '__main__':
    frame_number=0
    root=tk.Tk()
    heading = Label(root,bg='black')
    heading.pack()
    heading2=Label(root,text="Face To Emojify",pady=20, font=('arial',45,'bold'),bg='black',fg='#CDCDCD')#to label the output
    heading2.pack()
    lmain = tk.Label(master=root,padx=50,bd=10)
    lmain2 = tk.Label(master=root,bd=10)
    lmain3=tk.Label(master=root,bd=10,fg="#CDCDCD",bg='black')
    lmain.pack(side=LEFT)
    lmain.place(x=50,y=250)
    lmain3.pack()
    lmain3.place(x=960,y=250)
    lmain2.pack(side=RIGHT)
    lmain2.place(x=900,y=250)
    root.title("Face To Emojify")
    root.geometry("1400x900+100+10")
    root['bg']='black'
    exitbutton = Button(root, text='Quit',fg="red",command=root.destroy,font=('arial',25,'bold')).pack(side = BOTTOM)
    Capture_Image()
    Get_Emoji()
    root.mainloop()



1/1 [==============================] - 0s 20ms/step


In [2]:
# Import required Libraries
import tkinter as tk
from tkinter import *
from PIL import Image
from PIL import ImageTk
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D



# Build the convolution network architecture
face_model = Sequential()
face_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
face_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))
face_model.add(Flatten())
face_model.add(Dense(1024, activation='relu'))
face_model.add(Dropout(0.5))
face_model.add(Dense(7, activation='relu'))


# Load the saved weights
face_model.load_weights('recognition_model.h5')



# Disable OpenCL
cv2.ocl.setUseOpenCL(False)



# Create Datasets Dictionaries
facial_dict = {0: "   Angry   ", 1: "Disgusted", 2: "  Fearful  ", 3: "   Happy   ", 4: "  Neutral  ", 5: "    Sad    ", 6: "Surprised"}
#emojis_dict = {0:"train/angry", 1:"train/disgusted", 2:"train/fearful", 3:"train/happy", 4:"train/neutral", 5:"train/sad", 6:"train/surprised"}
emojis_dict = {
    0: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\angry.png",
    1: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\disgust.png",
    2: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\fear.png",
    3: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\happy.png",
    4: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\neutral.png",
    5: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\sad.png",
    6: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\surprise.png"
}



# Global variables
global last_frame1
last_frame1 = np.zeros((480, 640, 3), dtype=np.uint8)
global cap1
show_text=[0]



# Function to get face captured and recognize emotion
def Capture_Image():
    global cap1
    cap1 = cv2.VideoCapture(0)
    if not cap1.isOpened():
        print("cant open the camera1")
    global frame_number
    length =int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_number+=1
    if frame_number>=length:
        exit()
    cap1.set(1,frame_number)
    flag1, frame1 = cap1.read()
    frame1 = cv2.resize(frame1,(600,500))
    # It will detect the face in the video and bound it with a rectangular box
    #bound_box = cv2.CascadeClassifier('haarcascades_cuda/haarcascade_frontalface_default.xml')
    bounding_box = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    n_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in n_faces:
        cv2.rectangle(frame1, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_frame = gray_frame[y:y + h, x:x + w]
        crop_img = np.expand_dims(np.expand_dims(cv2.resize(roi_frame, (48, 48)), -1), 0)
        prediction = face_model.predict(crop_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame1, facial_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        show_text[0]=maxindex

    if flag1 is None:
        print ("Error!")

    elif flag1:
        global last_frame1
        last_frame1 = frame1.copy()
        pic = cv2.cvtColor(last_frame1, cv2.COLOR_BGR2RGB) #to store the image
        img = Image.fromarray(pic)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        lmain.after(10, Capture_Image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit()
        

# Function to show emoji according to facial expression
def Get_Emoji():
    frame2 = cv2.imread(emojis_dict[show_text[0]])
    if frame2 is None:
        print("Error: Could not load emoji image.")
        return
    pic2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
    img2 = Image.fromarray(pic2)
    imgtk2 = ImageTk.PhotoImage(image=img2)
    lmain2.imgtk2 = imgtk2
    lmain3.configure(text=facial_dict[show_text[0]], font=('arial', 45, 'bold'))
    lmain2.configure(image=imgtk2)
    root.update()
    lmain2.after(10, Get_Emoji)
if cv2.waitKey(1) & 0xFF == ord('q'):
    exit()
    
    
    


# GUI Window to show captured image with emoji
if __name__ == '__main__':
    frame_number=0
    root=tk.Tk()
    heading = Label(root,bg='black')
    heading.pack()
    heading2=Label(root,text="Face To Emojify",pady=20, font=('arial',45,'bold'),bg='black',fg='#CDCDCD')#to label the output
    heading2.pack()
    lmain = tk.Label(master=root,padx=50,bd=10)
    lmain2 = tk.Label(master=root,bd=10)
    lmain3=tk.Label(master=root,bd=10,fg="#CDCDCD",bg='black')
    lmain.pack(side=LEFT)
    lmain.place(x=50,y=250)
    lmain3.pack()
    lmain3.place(x=960,y=250)
    lmain2.pack(side=RIGHT)
    lmain2.place(x=900,y=250)
    root.title("Face To Emojify")
    root.geometry("1400x900+100+10")
    root['bg']='black'
    exitbutton = Button(root, text='Quit',fg="red",command=root.destroy,font=('arial',25,'bold')).pack(side = BOTTOM)
    Capture_Image()
    Get_Emoji()
    root.mainloop()



1/1 [==============================] - 0s 19ms/step


In [ ]:
# Import required Libraries
import tkinter as tk
from tkinter import *
from PIL import Image
from PIL import ImageTk
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D



# Build the convolution network architecture
face_model = Sequential()
face_model.add(Conv2D(32, kernel_size=(3, 3), activation='sigmoid', input_shape=(48,48,1)))
face_model.add(Conv2D(64, kernel_size=(3, 3), activation='sigmoid'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='sigmoid'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='sigmoid'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))
face_model.add(Flatten())
face_model.add(Dense(1024, activation='sigmoid'))
face_model.add(Dropout(0.5))
face_model.add(Dense(7, activation='sigmoid'))


# Load the saved weights
face_model.load_weights('recognition_model.h5')



# Disable OpenCL
cv2.ocl.setUseOpenCL(False)



# Create Datasets Dictionaries
facial_dict = {0: "   Angry   ", 1: "Disgusted", 2: "  Fearful  ", 3: "   Happy   ", 4: "  Neutral  ", 5: "    Sad    ", 6: "Surprised"}
#emojis_dict = {0:"train/angry", 1:"train/disgusted", 2:"train/fearful", 3:"train/happy", 4:"train/neutral", 5:"train/sad", 6:"train/surprised"}
emojis_dict = {
    0: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\angry.png",
    1: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\disgust.png",
    2: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\fear.png",
    3: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\happy.png",
    4: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\neutral.png",
    5: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\sad.png",
    6: r"C:\Users\mandl\OneDrive\Documents\PROJECT\MINI PROJECT-2\emojis\surprise.png"
}



# Global variables
global last_frame1
last_frame1 = np.zeros((480, 640, 3), dtype=np.uint8)
global cap1
show_text=[0]



# Function to get face captured and recognize emotion
def Capture_Image():
    global cap1
    cap1 = cv2.VideoCapture(0)
    if not cap1.isOpened():
        print("cant open the camera1")
    global frame_number
    length =int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_number+=1
    if frame_number>=length:
        exit()
    cap1.set(1,frame_number)
    flag1, frame1 = cap1.read()
    frame1 = cv2.resize(frame1,(600,500))
    # It will detect the face in the video and bound it with a rectangular box
    #bound_box = cv2.CascadeClassifier('haarcascades_cuda/haarcascade_frontalface_default.xml')
    bounding_box = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    n_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in n_faces:
        cv2.rectangle(frame1, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_frame = gray_frame[y:y + h, x:x + w]
        crop_img = np.expand_dims(np.expand_dims(cv2.resize(roi_frame, (48, 48)), -1), 0)
        prediction = face_model.predict(crop_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame1, facial_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        show_text[0]=maxindex

    if flag1 is None:
        print ("Error!")

    elif flag1:
        global last_frame1
        last_frame1 = frame1.copy()
        pic = cv2.cvtColor(last_frame1, cv2.COLOR_BGR2RGB) #to store the image
        img = Image.fromarray(pic)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        lmain.after(10, Capture_Image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit()
        

# Function to show emoji according to facial expression
def Get_Emoji():
    frame2 = cv2.imread(emojis_dict[show_text[0]])
    if frame2 is None:
        print("Error: Could not load emoji image.")
        return
    pic2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
    img2 = Image.fromarray(pic2)
    imgtk2 = ImageTk.PhotoImage(image=img2)
    lmain2.imgtk2 = imgtk2
    lmain3.configure(text=facial_dict[show_text[0]], font=('arial', 45, 'bold'))
    lmain2.configure(image=imgtk2)
    root.update()
    lmain2.after(10, Get_Emoji)
if cv2.waitKey(1) & 0xFF == ord('q'):
    exit()
    
    
    


# GUI Window to show captured image with emoji
if __name__ == '__main__':
    frame_number=0
    root=tk.Tk()
    heading = Label(root,bg='black')
    heading.pack()
    heading2=Label(root,text="Face To Emojify",pady=20, font=('arial',45,'bold'),bg='black',fg='#CDCDCD')#to label the output
    heading2.pack()
    lmain = tk.Label(master=root,padx=50,bd=10)
    lmain2 = tk.Label(master=root,bd=10)
    lmain3=tk.Label(master=root,bd=10,fg="#CDCDCD",bg='black')
    lmain.pack(side=LEFT)
    lmain.place(x=50,y=250)
    lmain3.pack()
    lmain3.place(x=960,y=250)
    lmain2.pack(side=RIGHT)
    lmain2.place(x=900,y=250)
    root.title("Face To Emojify")
    root.geometry("1400x900+100+10")
    root['bg']='black'
    exitbutton = Button(root, text='Quit',fg="red",command=root.destroy,font=('arial',25,'bold')).pack(side = BOTTOM)
    Capture_Image()
    Get_Emoji()
    root.mainloop()



1/1 [==============================] - 0s 32ms/step
